In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import glob
import random
from tensorflow.keras.layers import Flatten, Dense,Dropout
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import BatchNormalization

In [2]:
DATADIR = 'dataset/'

In [3]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
IMG_SIZE = 300
train = image_dataset_from_directory(DATADIR,
                                     labels="inferred",
                                     label_mode="categorical",
                                     color_mode="rgb",
                                     batch_size=256,
                                     image_size=(
                                         IMG_SIZE, IMG_SIZE),
                                     shuffle=True,
                                     seed=777,
                                     validation_split=0.2,
                                     subset="training")
val = image_dataset_from_directory(DATADIR,
                                   labels="inferred",
                                   label_mode="categorical",
                                   color_mode="rgb",
                                   batch_size=256,
                                   image_size=(
                                       IMG_SIZE, IMG_SIZE),
                                   shuffle=True,
                                   seed=777,
                                   validation_split=0.2,
                                   subset="validation")
test = train.skip(int(0.75*len(train)))
train = train.take(int(0.75*len(train)))
test = test.take(int(0.25*len(train)))

Found 10716 files belonging to 150 classes.
Using 8573 files for training.
Found 10716 files belonging to 150 classes.
Using 2143 files for validation.


In [4]:
train=train.repeat(count=7)
train=train.shuffle(buffer_size=128)
len(train)

175

In [5]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.4),
        layers.experimental.preprocessing.RandomZoom(0.5),
        layers.experimental.preprocessing.RandomContrast(0.3),
        layers.experimental.preprocessing.Rescaling(1./255)
    ]
)

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train = train.prefetch(buffer_size=AUTOTUNE)
val = val.prefetch(buffer_size=AUTOTUNE)
test = test.prefetch(buffer_size=AUTOTUNE)

In [7]:
vgg_model = VGG16(weights='imagenet', include_top=False,
                  input_shape=(IMG_SIZE, IMG_SIZE, 3))
vgg_model.trainable = True

trainable_layer = 3
for layer in vgg_model.layers[:-trainable_layer]:
    layer.trainable = False
for layer in vgg_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f19a3729c40> False
<keras.layers.convolutional.Conv2D object at 0x7f19a38003a0> False
<keras.layers.convolutional.Conv2D object at 0x7f19a3800b80> False
<keras.layers.pooling.MaxPooling2D object at 0x7f19a3729d60> False
<keras.layers.convolutional.Conv2D object at 0x7f19a38832b0> False
<keras.layers.convolutional.Conv2D object at 0x7f19a389cf70> False
<keras.layers.pooling.MaxPooling2D object at 0x7f19a389cc70> False
<keras.layers.convolutional.Conv2D object at 0x7f19a38a23a0> False
<keras.layers.convolutional.Conv2D object at 0x7f19a38a2f10> False
<keras.layers.convolutional.Conv2D object at 0x7f19a3880ac0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f19a3883fa0> False
<keras.layers.convolutional.Conv2D object at 0x7f19a38a95e0> False
<keras.layers.convolutional.Conv2D object at 0x7f19a38ae4f0> False
<keras.layers.convolutional.Conv2D object at 0x7f19a388ddc0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f19a38a20a0>

In [8]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# Image augmentation block
x = data_augmentation(inputs)
x = vgg_model(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
outputs = Dense(150, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 300, 300, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 9, 9, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 1024)              42468352  
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                             

In [9]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f19a38a9a00> True
<keras.engine.sequential.Sequential object at 0x7f19a4545a30> True
<keras.engine.functional.Functional object at 0x7f19a38bf310> True
<keras.layers.core.flatten.Flatten object at 0x7f19a38a97c0> True
<keras.layers.core.dense.Dense object at 0x7f19a38bdfa0> True
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f19a36e5b20> True
<keras.layers.core.dropout.Dropout object at 0x7f19a3800d60> True
<keras.layers.core.dense.Dense object at 0x7f19a3800e80> True
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f19a36e5820> True
<keras.layers.core.dropout.Dropout object at 0x7f19a0df4970> True
<keras.layers.core.dense.Dense object at 0x7f19a0e0a640> True


In [10]:
learning_rate = 1e-4
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [11]:
model_dir = 'model-logs/mid/vgg16'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

modelfiles = model_dir + '/{}-best-model.h5'.format('basic1_model')
model_mckp = keras.callbacks.ModelCheckpoint(modelfiles,
                                             monitor='val_accuracy',
                                             save_best_only=True)

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                          patience=10,
                                          verbose=1)

callbacks_list = [model_mckp, earlystop]

In [12]:
history = model.fit(
    train,
    batch_size=256,
    epochs=50,
    validation_data=val,
    callbacks=callbacks_list
)

Epoch 1/50
175/175 [==============================] - 164s 652ms/step - loss: 3.6481 - accuracy: 0.2502 - val_loss: 2.5635 - val_accuracy: 0.4251
Epoch 2/50
175/175 [==============================] - 148s 641ms/step - loss: 2.0717 - accuracy: 0.5221 - val_loss: 1.7616 - val_accuracy: 0.5903
Epoch 3/50
175/175 [==============================] - 148s 643ms/step - loss: 1.4528 - accuracy: 0.6612 - val_loss: 1.5392 - val_accuracy: 0.6360
Epoch 4/50
175/175 [==============================] - 148s 641ms/step - loss: 1.1081 - accuracy: 0.7351 - val_loss: 1.4172 - val_accuracy: 0.6547
Epoch 5/50
175/175 [==============================] - 147s 639ms/step - loss: 0.8838 - accuracy: 0.7862 - val_loss: 1.3070 - val_accuracy: 0.6804
Epoch 6/50
175/175 [==============================] - 147s 641ms/step - loss: 0.7284 - accuracy: 0.8202 - val_loss: 1.2173 - val_accuracy: 0.6920
Epoch 7/50
175/175 [==============================] - 148s 642ms/step - loss: 0.5986 - accuracy: 0.8539 - val_loss: 1.2387 -

In [13]:
model.load_weights(filepath=modelfiles)
score = model.evaluate(test, verbose=1)
print("\n Accuracy:{}".format(score[1]))

6/6 [==============================] - 10s 566ms/step - loss: 0.3515 - accuracy: 0.9076

 Accuracy:0.9075520634651184
